In [ ]:
import matplotlib.pyplot as plt
import pandas, sys
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

Load data in, we will be selecting aggregation propensities measured using the Martini 2.1 forcefield for dipeptides

In [ ]:
#Load the parameters we just created with the Judred script
parameters = pandas.read_csv("Dipeptides_Judred.csv", index_col=0)

#Load the AP scores which are stored in APs.csv
targets = pandas.read_csv("APs.csv", index_col = 0)

#This csv contains more than the data we are looking to match to these specific parameters so firstly,
#we filter for just the AP score for Martini 2.1
Forcefield = "2.1"
targets = targets[targets["FF"] == Forcefield]
#We want this dataframe to have the same index labels as our parameters, currently parameters.index = ["AA", "AC", ...]
#while targets.index = ["AA_2.1", "AC_2.1", ...] so we will change it to the 'pep' column which sets targets.index = ["AA", "AC", ...]
targets.index = targets["pep"]

#Finally we will take the mean value from the AP scores measure over triplicate runs
targets = targets["mean"]
print(targets)

We split these into training, testing and validation sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(parameters, targets, test_size=0.33, random_state=9876, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=9876, shuffle=True)

useRBF = False

print("Training data:", X_train.shape)
print("Testing data:", X_test.shape)
print("Validation data:", X_val.shape)

We will take a look at how projecting our data into other spaces could improve our predictions. But we will uncomment this at the end to determine its effect.

In [ ]:
a="""
X_train, X_val, y_train, y_val = train_test_split(parameters, targets, test_size=0.33, random_state=9876, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=9876, shuffle=True)

# Add Feature Transformation
def rbf(X, epsilon):
    return np.e ** -(epsilon*X)**2

X_train_RBF = np.hstack((X_train.values, rbf(X_train, 0.2)))
X_test_RBF = np.hstack((X_test.values, rbf(X_test, 0.2)))
X_val_RBF = np.hstack((X_val.values, rbf(X_val, 0.2)))

y_train_RBF = y_train
y_test_RBF = y_test
y_val_RBF = y_val
print("Training data:", X_train_RBF.shape)
print("Testing data:", X_test_RBF.shape)
print("Validation data:", X_val_RBF.shape)

useRBF = True
#"""

Test all the combinations of the hyperparameters on the test set to see what works best for our model

In [ ]:
BestHyperparameters = pandas.DataFrame(columns = ["fit_intercept", "positive", "Test data RMSE"])
iteration = 0
for fit_intercept in [True, False]:
    for positive in [True, False]:
        Hyperparameters = {"fit_intercept": fit_intercept,
                           "positive": positive}
        
        model = LinearRegression(**Hyperparameters)
        
        
        if useRBF:
            #Train on the training data
            model.fit(X_train_RBF, y_train_RBF)
            predictions = model.predict(X_test_RBF)
        else:
            #Train on the training data
            model.fit(X_train, y_train)        
            # Test on the Test data
            predictions = model.predict(X_test)
        
        BestHyperparameters.loc[iteration] = [fit_intercept, positive, mean_squared_error(y_test, predictions, squared=False)]
        iteration +=1
BestHyperparameters = BestHyperparameters.sort_values("Test data RMSE", ascending=False)
print(BestHyperparameters)



Take our best hyperparameters and make a prediction on the validation set

In [ ]:
if useRBF:
    try:
        epsilon = BestHyperparameters.iloc[-1]["epsilon"]
    except KeyError:
        print("You have not added an 'epsilon' column to the BestHyperparameters dataframe! Reverting to 0.2")
        epsilon = 0.2
        X_train = np.hstack((X_train.values, rbf(X_train, epsilon)))
        X_test = np.hstack((X_test.values, rbf(X_test, epsilon)))
        X_val = np.hstack((X_val.values, rbf(X_val, epsilon)))

    

Hyperparameters = {"fit_intercept": BestHyperparameters.iloc[-1]["fit_intercept"],
                   "positive": BestHyperparameters.iloc[-1]["positive"]}
model = LinearRegression(**Hyperparameters)
model.fit(X_train, y_train)

# Validate on the never before seen in any way validation data
predictions = model.predict(X_val)
RMSE = mean_squared_error(y_val, predictions, squared=False)

In [ ]:
plt.scatter(predictions, y_val)
plt.plot([1,2.7], [1,2.7], lw=1, c="black")
plt.title(f"RMSE = " + "{:1.3f}".format(RMSE))
plt.xlabel("Predicted AP")
plt.ylabel("True AP")
plt.gcf().set_dpi(150)
plt.show()

# Now, go back and add the kernel trick to the data